# Import

In [ ]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.time import RandomActivationByType
from mesa.datacollection import DataCollector
import random
from mesa import Agent
from numpy import random
from mesa.visualization import CanvasGrid, ModularServer
import numpy as np
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
import matplotlib.pyplot as plt

In [ ]:
import solara
from matplotlib.figure import Figure
import random

In [ ]:
def get_distance(pos1, pos2):
    return (pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2

# Class cell

In [ ]:
class Cell(Agent):
    def __init__(self, unique_id, model, capacities):
        super().__init__(unique_id, model)
        self.capacities = capacities
        self.sugar = capacities[0]
        self.spice = capacities[1]

    def step(self):
        self.regenerate()

    def regenerate(self):
        self.sugar = min(self.sugar + 1, self.capacities[0])
        self.spice = min(self.spice + 1, self.capacities[1])

# Class Trader

In [ ]:
class Trader(Agent):
    def __init__(self, unique_id, model, sugar, sugar_metabolism, spice, spice_metabolism, vision, max_age):
        super().__init__(unique_id, model)
        self.sugar = sugar
        self.sugar_metabolism = sugar_metabolism
        self.spice = spice
        self.spice_metabolism = spice_metabolism
        self.vision = vision
        self.max_age = max_age
        self.spice_weight = sugar_metabolism / (sugar_metabolism + spice_metabolism)
        self.sugar_weight = 1 - self.spice_weight
        self.age = 0

        # Set initial wealth
        self.wealth = 0
        self.update_wealth()

        self.repopulated = False

    def step(self):
        self.move()
        #print(self.repopulated)
        if self.sugar > 20 and self.spice > 20 and random.random() < 0.05 and self.repopulated == False:
            self.repopulate()
        self.pick_up()
        self.update_wealth()
        self.trade()
        self.metabolize()
        self.get_old()

    def move(self):
        # Get neighborhood
        neighbours = [i for i in self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False, radius=self.vision)]
        max_total = -1
        shortest_distance = float('inf')
        max_cell = []

        # Get cell with most sugar
        for neighbour in neighbours:
            this_cell = self.model.grid.get_cell_list_contents([neighbour])
            for agent in this_cell:
                if isinstance(agent, Cell):
                    # Compute weighted average of sugar and spice
                    weighted_sugar = self.sugar_weight * agent.sugar
                    weighted_spice = self.spice_weight * agent.spice
                    total = weighted_sugar + weighted_spice

                    if total > max_total or (total == max_total and get_distance(self.pos, neighbour) < shortest_distance):
                        max_total = total
                        shortest_distance = get_distance(self.pos, neighbour)
                        max_cell = [neighbour]
                    elif total == max_total and get_distance(self.pos, neighbour) == shortest_distance:
                        max_cell.append(neighbour)

        if len(max_cell) > 0:
            new_position = random.choice(max_cell)
            self.model.grid.move_agent(self, new_position)

    def pick_up(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        # Grab all sugar and spice from cell
        for agent in this_cell:
            if isinstance(agent, Cell):
                self.sugar += agent.sugar
                agent.sugar = 0

                self.spice += agent.spice
                agent.spice = 0

    def metabolize(self):
        # Metabolize sugar
        self.sugar -= self.sugar_metabolism
        # Metabolize spice
        self.spice -= self.spice_metabolism
        
        # Die if sugar or spice is less than 0
        if self.sugar < 0 or self.spice < 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            self.model.deaths_by_hunger_step += 1

    def get_old(self):

        self.age += 1

        # Die if meets max_age
        if self.age >= self.max_age:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            self.model.deaths_by_age_step += 1

    def repopulate(self):
        true_neighbours = []
        fitness = []
        neighbors = self.model.grid.get_neighbors(self.pos, moore=False, include_center=False, radius=1)
        for neighbor in neighbors:
            if isinstance(neighbor, Trader):
                true_neighbours.append(neighbor)
                fitness.append(neighbor.sugar + neighbor.spice)
        if len(true_neighbours) == 0:
            return
        partner = roulette_wheel_selection(true_neighbours, fitness)
        sugar_metabolism = random.choice([self.sugar_metabolism, partner.sugar_metabolism])
        spice_metabolism = random.choice([self.spice_metabolism, partner.spice_metabolism])
        sugar, spice = np.random.randint(1, 11, 2)
        vision = random.choice([self.vision, partner.vision])
        max_age = random.choice([self.max_age, partner.max_age])
        new_trader = Trader(self.model.schedule.get_agent_count() + 1, self.model, sugar, sugar_metabolism, spice, spice_metabolism, vision, max_age)
        mutate(new_trader, 0.3)
        positions = self.model.grid.get_neighbors(self.pos, moore=False, include_center=False, radius=1)
        cells = [pos for pos in positions if isinstance(pos, Cell)]
        # print("repopulating", len(cells))
        if len(cells) > 0:
            selected_pos = random.choice(cells)
            self.model.grid.place_agent(new_trader, selected_pos.pos)
            self.model.schedule.add(new_trader)
            self.repopulated = True


    def trade(self):
        neighbors = self.model.grid.get_neighbors(self.pos, moore=False, include_center=False, radius=1)
        random.shuffle(neighbors)
        for neighbor in neighbors:
            if isinstance(neighbor, Trader):
                while True:
                    my_mrs = self.get_mrs_sugar_spice()
                    their_mrs = neighbor.get_mrs_sugar_spice()

                    if my_mrs == their_mrs:
                        break

                    if my_mrs > their_mrs:
                        trader_high_mrs = self
                        trader_low_mrs = neighbor
                    else:
                        trader_high_mrs = neighbor
                        trader_low_mrs = self

                    trade_price = np.sqrt(my_mrs * their_mrs)
                    if trade_price > 1:
                        trade_spice = trade_price
                        trade_sugar = 1
                    else:
                        trade_spice = 1
                        trade_sugar = 1 / trade_price

                    trade_sugar = min(trade_sugar, trader_low_mrs.sugar)
                    trade_spice = min(trade_spice, trader_high_mrs.spice)

                    if trade_sugar <= 0 or trade_spice <= 0:
                        break

                    if self.improve_welfare(trader_high_mrs, trader_low_mrs, trade_sugar, trade_spice):
                        trader_high_mrs.spice -= trade_spice
                        trader_high_mrs.sugar += trade_sugar
                        trader_low_mrs.spice += trade_spice
                        trader_low_mrs.sugar -= trade_sugar

                        self.model.datacollector.add_table_row("Trades", {
                            'Step': self.model.current_step,
                            'TraderHighMRS_ID': trader_high_mrs.unique_id,
                            'TraderLowMRS_ID': trader_low_mrs.unique_id,
                            'TradeSugar': trade_sugar,
                            'TradeSpice': trade_spice,
                            'TradePrice': trade_price
                        })
                    else:
                        break 

    def get_mrs_sugar_spice(self):
        return (self.sugar_metabolism * self.spice) / (self.spice_metabolism * self.sugar + 1e-9)

    def improve_welfare(self, trader_high_mrs, trader_low_mrs, trade_sugar, trade_spice):
        high_mrs_after_trade = (trader_high_mrs.sugar_metabolism * (trader_high_mrs.spice - trade_spice)) / (trader_high_mrs.spice_metabolism * (trader_high_mrs.sugar + trade_sugar + 1e-9))
        low_mrs_after_trade = (trader_low_mrs.sugar_metabolism * (trader_low_mrs.spice + trade_spice)) / (trader_low_mrs.spice_metabolism * (trader_low_mrs.sugar - trade_sugar + 1e-9))
        improves_welfare = high_mrs_after_trade < trader_high_mrs.get_mrs_sugar_spice() and low_mrs_after_trade > trader_low_mrs.get_mrs_sugar_spice()
        mrs_no_crossing = high_mrs_after_trade > low_mrs_after_trade
        return improves_welfare and mrs_no_crossing
    
    def welfare(self, sugar, spice):
        return sugar ** self.sugar_weight * spice ** self.spice_weight

    def update_wealth(self):
        self.wealth = self.welfare(self.sugar, self.spice)

def roulette_wheel_selection(population, fitness):
    total_fitness = sum(fitness)
    selection_probs = [f / total_fitness for f in fitness]
    return population[np.random.choice(range(len(population)), p=selection_probs)]

def mutate(agent, mutation_rate):
    if random.random() < mutation_rate:
        agent.sugar_metabolism = np.random.normal(agent.sugar_metabolism, agent.sugar_metabolism*0.2)
        if agent.sugar_metabolism < 0:
            agent.sugar_metabolism = 0
    if random.random() < mutation_rate:
        agent.spice_metabolism = np.random.normal(agent.spice_metabolism, agent.spice_metabolism*0.2)
        if agent.spice_metabolism < 0:
            agent.spice_metabolism = 0
    if random.random() < mutation_rate:
        if random.random() < 0.5:
            agent.vision = agent.vision + 1
        else:
            agent.vision = agent.vision - 1
        if agent.vision < 1:
            agent.vision = 1


# Class Distributer

## BaseDistributer

In [ ]:
class BaseDistributer:
    def __init__(self, distributer_steps) -> None:
        self.distributer_steps = distributer_steps
        self.current_step = 0

    def step(self, agents, taxer):
        self.current_step += 1
        if self.current_step % self.distributer_steps == 0:
            self.distribute(agents, taxer)

    def distribute(self, agents, taxer):
        total_agents = len([agent for agent in agents if isinstance(agent, Trader)])
        if total_agents > 0:
            sugar_per_agent = taxer.taxes_collection["sugar"] / total_agents
            spice_per_agent = taxer.taxes_collection["spice"] / total_agents
            for agent in agents:
                agent.sugar += sugar_per_agent
                agent.spice += spice_per_agent

            # Reset taxes collection
            taxer.reset_tax()

## ProgressiveDistributer

In [ ]:
class ProgressiveDistributer(BaseDistributer):
    def distribute(self, agents, taxer):
        # Get wealth of all agents
        wealths = [agent.wealth for agent in agents]
        wealths.sort()

        # Find the threshold for classes
        low_n = len(wealths) // 3 + 1
        middle_n = 2 * low_n
        low_class_threshold = wealths[low_n]
        middle_class_threshold = wealths[middle_n]

        # Find how much each class gets distributed
        low_class = {}
        middle_class = {}
        high_class = {}
        for key in taxer.taxes_collection:
            # Compute how much middle class gets
            middle_class[key] = taxer.taxes_collection[key] / (7 / 3 * low_n + 2 / 3 * (len(wealths) - middle_n))

            # Compute how much low and high class gets
            low_class[key] = middle_class[key] * 4 / 3
            high_class[key] = middle_class[key] * 2 / 3

        # Distribute
        for agent in agents:
            if agent.wealth < low_class_threshold:
                agent.sugar += low_class["sugar"]
                agent.spice += low_class["spice"]

            elif agent.wealth < middle_class_threshold:
                agent.sugar += middle_class["sugar"]
                agent.spice += middle_class["spice"]

            else:
                agent.sugar += high_class["sugar"]
                agent.spice += high_class["spice"]

        # Reset taxes collection
        taxer.reset_tax()

# Class Tax

## BaseTaxer

In [ ]:
class BaseTaxer:
    def __init__(self, tax_steps, tax_rate) -> None:
        self.tax_steps = tax_steps
        self.tax_rate = tax_rate
        self.taxes_collection = {"sugar": 0, "spice": 0}
        self.current_step = 0

    def step(self, agents):
        self.current_step += 1
        if self.current_step % self.tax_steps == 0:
            self.collect_taxes(agents)

    def collect_taxes(self, agents):
        for agent in agents:
            # Compute tax
            sugar_tax = agent.sugar * self.tax_rate
            spice_tax = agent.spice * self.tax_rate

            # Update agent's goods and taxes collection
            agent.sugar -= sugar_tax
            agent.spice -= spice_tax
            self.taxes_collection["sugar"] += sugar_tax
            self.taxes_collection["spice"] += spice_tax

    def reset_tax(self):
        self.taxes_collection = {"sugar": 0, "spice": 0}

## ProgressiveTaxer

In [ ]:
class ProgressiveTaxer(BaseTaxer):
    def collect_taxes(self, agents):
        # Get wealth distribution to determine tax rates
        wealths = [agent.wealth for agent in agents]

        # Sort wealth
        wealths.sort()

        # Find 33rd and 66th percentiles
        low_class = wealths[len(wealths) // 3]
        middle_class = wealths[2 * len(wealths) // 3]

        # Collect taxes
        for agent in agents:
            if agent.wealth < low_class:
                self.update_goods(agent, self.tax_rate * 0.66)
            elif agent.wealth < middle_class:
                self.update_goods(agent, self.tax_rate)
            else:
                self.update_goods(agent, self.tax_rate * 1.33)

    def update_goods(self, agent, tax_rate):
        self.taxes_collection["sugar"] += int(agent.sugar * tax_rate)
        self.taxes_collection["spice"] += int(agent.spice * tax_rate)
        agent.sugar -= int(agent.sugar * tax_rate)
        agent.spice -= int(agent.spice * tax_rate)

# Class SugarScape

## Global functions

In [ ]:
def compute_trade_counts(model):
    trade_data = model.get_trade_log()
    current_step_trades = trade_data[trade_data["Step"] == model.current_step]
    return len(current_step_trades)

def compute_average_trade_price(model):
    trade_data = model.get_trade_log()
    if len(trade_data) == 0:
        return 0
    current_step_trades = trade_data[trade_data["Step"] == model.current_step]
    if len(current_step_trades) == 0:
        return 0
    average_price = current_step_trades["TradePrice"].mean()
    return average_price

def compute_gini(model):
    agent_wealths = [agent.sugar/agent.sugar_metabolism + agent.spice/agent.spice_metabolism for agent in model.schedule.agents if isinstance(agent, Trader)]
    sorted_wealths = sorted(agent_wealths)
    # plt.hist(sorted_wealths, bins=10)
    # plt.show()
    n = len(sorted_wealths)
    #print(n)
    if n == 0:
        return 0
    cumulative_sum = sum((i + 1) * wealth for i, wealth in enumerate(sorted_wealths))
    total_wealth = sum(sorted_wealths)
    gini = (2 * cumulative_sum) / (n * total_wealth) - (n + 1) / n
    
    return gini

def compute_deaths_by_age(model):
    """Return the number of deaths by age for the current step."""
    return model.deaths_by_age[-1] if model.deaths_by_age else 0

def compute_deaths_by_hunger(model):
    """Return the number of deaths by hunger for the current step."""
    return model.deaths_by_hunger[-1] if model.deaths_by_hunger else 0

def compute_average_vision(model):
    """Compute the average vision of all living Trader agents."""
    traders = [agent for agent in model.schedule.agents if isinstance(agent, Trader)]
    if len(traders) == 0:
        return 0
    average_vision = sum(trader.vision for trader in traders) / len(traders)
    return average_vision

def compute_average_sugar_metabolism(model):
    """Compute the average sugar metabolism of all living Trader agents."""
    traders = [agent for agent in model.schedule.agents if isinstance(agent, Trader)]
    if len(traders) == 0:
        return 0
    average_sugar_metabolism = sum(trader.sugar_metabolism for trader in traders) / len(traders)
    return average_sugar_metabolism

def compute_average_spice_metabolism(model):
    """Compute the average spice metabolism of all living Trader agents."""
    traders = [agent for agent in model.schedule.agents if isinstance(agent, Trader)]
    if len(traders) == 0:
        return 0
    average_spice_metabolism = sum(trader.spice_metabolism for trader in traders) / len(traders)
    return average_spice_metabolism


## Main part

In [ ]:
class SugarScape(Model):
    def __init__(self, height=50, width=50, initial_population=100,
                 tax_scheme="flat", tax_steps=10, tax_rate=0.1, 
                 distributer_scheme="flat", distributer_steps=20,seed_value=42):
        super().__init__()
        self.height = height
        self.width = width
        self.current_step = 0
        self.initial_population = initial_population
        self.population = initial_population

        # Set seed for reproducibility
        random.seed(seed_value)

        self.deaths_by_age = []
        self.deaths_by_hunger = []
        self.deaths_by_age_step = 0
        self.deaths_by_hunger_step = 0
        
        # Create taxers and distributers
        if tax_scheme == "flat":
            self.taxer = BaseTaxer(tax_steps, tax_rate)
        elif tax_scheme == "progressive":
            self.taxer = ProgressiveTaxer(tax_steps, tax_rate)
        else:
            raise ValueError("Invalid tax scheme")

        if distributer_scheme == "flat":
            self.distributer = BaseDistributer(distributer_steps)
        elif distributer_scheme == "progressive":
            self.distributer = ProgressiveDistributer(distributer_steps)
        else:
            raise ValueError("Invalid distributer scheme")
        
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(self.height, self.width, False)

        
         # Create cells
        id = 0
        for content, (x, y) in self.grid.coord_iter():
            # Define capacities and reproduction rates based on location
            if x < self.width // 2 and y < self.height // 2:  # Left Upper
                capacities = [random.randint(5, 10), random.randint(0, 2)]
            elif x < self.width // 2 and y >= self.height // 2:  # Left Lower
                capacities = [random.randint(5, 10), random.randint(0, 2)]
            elif x >= self.width // 2 and y < self.height // 2:  # Right Upper
                capacities = [random.randint(0, 2), random.randint(5, 10)]
            else:  # Right Lower
                capacities = [random.randint(0, 2), random.randint(5, 10)]

            cell = Cell(id, self, capacities)

            # Place cell on grid
            self.grid.place_agent(cell, (x, y))
            self.schedule.add(cell)

            # Increment id
            id += 1

        # Create traders
        self.traders = {}
        for i in range(self.initial_population):
            # Random position
            x = random.randint(0, self.width-1)
            y = random.randint(0, self.height-1)

            # Instantiate trader
            sugar, spice = np.random.randint(1, 10, 2)
            sugar_metabolism, spice_metabolism = np.random.randint(1, 4, 2)
            vision = random.randint(1, 4)
            max_age = random.randint(70, 100)
            trader = Trader(id, self, sugar, sugar_metabolism, spice, spice_metabolism, vision, max_age)

            # Place trader on grid
            self.grid.place_agent(trader, (x, y))
            self.schedule.add(trader)

            # Add trader to dictionary
            self.traders[id] = trader

            # Increment id
            id += 1

        self.datacollector = DataCollector(
            model_reporters={
                "Trade Price": compute_average_trade_price,
                "Gini": compute_gini,
                "Number of Trades":compute_trade_counts,
                "Deaths by Age": compute_deaths_by_age,
                "Deaths by Hunger": compute_deaths_by_hunger,
                "Average Vision": compute_average_vision,
                "Average Sugar Metabolism": compute_average_sugar_metabolism,
                "Average Spice Metabolism": compute_average_spice_metabolism,
            },
            tables={"Trades": ["Step", "TraderHighMRS_ID", "TraderLowMRS_ID", "TradeSugar", "TradeSpice", "TradePrice"]}
        )

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        self.current_step += 1

        # Get all trader
        traders = [agent for agent in self.schedule.agents if isinstance(agent, Trader)]

        # Take step for taxer and distributer
        self.taxer.step(traders)
        self.distributer.step(traders, self.taxer)
        
        # Append the number of deaths in this step to the lists
        self.deaths_by_age.append(self.deaths_by_age_step)
        self.deaths_by_hunger.append(self.deaths_by_hunger_step)

        # Reset the step death counters
        self.deaths_by_age_step = 0
        self.deaths_by_hunger_step = 0

        self.datacollector.collect(self)
        self.running = True if self.schedule.get_agent_count() > 0 else False

    def run_model(self, step_count=200):
        for i in range(step_count):
            self.step()


    def get_trade_log(self):
        return self.datacollector.get_table_dataframe("Trades")


# Run model

In [ ]:
SugarScape().run_model()

In [ ]:

def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {"Filled": "true",
                 "r": 0.5,
                 "w": 1,
                 "h": 1}

    if type(agent) is Trader:
        portrayal["Color"] = "red"
        portrayal["Layer"] = 1
        portrayal["Shape"] = "circle"
    elif type(agent) is Cell:
        portrayal["Shape"] = "rect"
        portrayal["Color"] = "green" if agent.sugar > 0 and agent.spice > 0 else "black"
        portrayal["Layer"] = 0

    return portrayal
canvas_element = CanvasGrid(agent_portrayal, 50, 50, 500, 500)

trade_count_chart = ChartModule(
    [{"Label": "Number of Trades", "Color": "Blue"}],
    data_collector_name='datacollector'
)

average_trade_price_chart = ChartModule(
    [{"Label": "Trade Price", "Color": "Red"}],
    data_collector_name='datacollector'
)

gini_pop = ChartModule(
    [{"Label": "Gini", "Color": "Black"}],
    data_collector_name='datacollector'
)

deaths_by_age_chart = ChartModule(
    [{"Label": "Deaths by Age", "Color": "Green"}],
    data_collector_name='datacollector'
)

deaths_by_hunger_chart = ChartModule(
    [{"Label": "Deaths by Hunger", "Color": "Orange"}],
    data_collector_name='datacollector'
)

average_vision_chart = ChartModule(
    [{"Label": "Average Vision", "Color": "Purple"}],
    data_collector_name='datacollector'
)

average_sugar_metabolism_chart = ChartModule(
    [{"Label": "Average Sugar Metabolism", "Color": "Pink"}],
    data_collector_name='datacollector'
)

average_spice_metabolism_chart = ChartModule(
    [{"Label": "Average Spice Metabolism", "Color": "Brown"}],
    data_collector_name='datacollector'
)

server = ModularServer(
    SugarScape, 
    [canvas_element, trade_count_chart, average_trade_price_chart, gini_pop,
     deaths_by_age_chart,deaths_by_hunger_chart,average_vision_chart,
     average_sugar_metabolism_chart,average_spice_metabolism_chart], 
    "Sugarscape Model",
    {"height": 50, "width": 50, "initial_population": 100}
    
)

server.port = 8562
server.launch()